Monter le drive dans google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import des librairies

In [ ]:
import pandas as pd
import numpy as np
import datetime

Import des fichiers df, df_by_team au format csv dans google colab : si besoin, changer le chemin /content/...

In [ ]:
# Gestion des dates au format date avec format européen
df= pd.read_csv('/content/df.csv', parse_dates=['Date'],dayfirst=True,index_col=0)
df_by_team= pd.read_csv('/content/df_by_team.csv', parse_dates=['Date'],dayfirst=True,index_col=0)

# La saison lue en tant que numérique est remise en chaine de caractère
df['Saison'] = df['Saison'] .astype(str)
df_by_team['Saison'] = df_by_team['Saison'] .astype(str)

# Import d'un csv contenant les notes FIFA (jeu) par équipe et saison
df_note = pd.read_csv('/content/note_equipe.csv', sep=';') 
df_note["Saison"]=df_note["Saison"].astype(str)

## Feature engineering

Ajout du résultat du même match la saison précédante

In [ ]:
# On ne garde que les variables qui serviront soit à la fusion, soit à l'ajout de variables qui nous interessent (résultat, nb de buts)
df_result_last_an = df[['Equipe_exterieur','Equipe_domicile','Saison','Resultat_match','Nb_buts_equipe_dom','Nb_buts_equipe_ext']]
df_result_last_an=df_result_last_an.rename(columns={'Resultat_match' : 'Resultat_match_an_dernier',
                                  'Nb_buts_equipe_dom':'Nb_buts_equipe_dom_an_dernier',
                                  'Nb_buts_equipe_ext':'Nb_buts_equipe_ext_an_dernier'})

# Ajout de 1 à la saison pour préparation de la fusion
df_result_last_an.loc[:,'Saison']=(df_result_last_an.loc[:,'Saison'].astype(int)+1).astype(str)

# Fusion du dataframe de départ avec les résultats de l'an dernier
df = df.merge(right = df_result_last_an, on = ['Equipe_domicile','Equipe_exterieur','Saison'], how = 'left')

# Remplacement des valeurs manquantes par 0 pour les nombres de buts
df['Nb_buts_equipe_dom_an_dernier'].fillna(0,inplace=True)
df['Nb_buts_equipe_ext_an_dernier'].fillna(0,inplace=True)

In [ ]:
# Extrait du dataframe
df.sample(10)

,Date,Saison,Equipe_domicile,Equipe_exterieur,Nb_buts_equipe_dom,Nb_buts_equipe_ext,Resultat_match,Nb_buts_mi_temps_equipe_dom,Nb_buts_mi_temps_equipe_ext,Resultat_mi_temps,Nb_tirs_equipe_dom,Nb_tirs_equipe_ext,Nb_tirs_cadres_equipe_dom,Nb_tirs_cadres_equipe_ext,Nb_fautes_equipe_dom,Nb_fautes_equipe_ext,Corners_equipe_dom,Corners_equipe_ext,Nb_cartons_J_equipe_dom,Nb_cartons_J_equipe_ext,Nb_cartons_R_equipe_dom,Nb_cartons_R_equipe_ext,Nb_total_but_match,Moy_cote_domicile,Max_cote_domicile,Min_cote_domicile,Moy_cote_exterieur,Max_cote_exterieur,Min_cote_exterieur,Moy_cote_nul,Max_cote_nul,Min_cote_nul,Resultat_match_an_dernier,Nb_buts_equipe_dom_an_dernier,Nb_buts_equipe_ext_an_dernier
2389,2022-01-23,2021,Metz,Nice,0,2,A,0,0,D,8,7,1,6,14,6,3,4,3,1,0,0,2,4.96,5.25,4.60,1.72,1.75,1.70,3.72,3.90,3.50,D,1.0,1.0
1313,2018-12-22,2018,Reims,Caen,2,2,D,1,2,A,19,7,11,2,9,16,10,2,3,3,0,0,4,1.74,1.80,1.70,5.50,5.92,4.75,3.45,3.50,3.40,NaN,0.0,0.0
805,2017-09-09,2017,Troyes,Toulouse,0,0,D,0,0,D,11,7,3,0,12,13,4,1,2,3,0,0,0,2.88,2.98,2.80,2.55,2.60,2.50,3.19,3.28,3.10,NaN,0.0,0.0
1843,2020-09-27,2020,Dijon,Montpellier,2,2,D,1,0,H,15,14,6,5,15,11,6,13,4,1,0,0,4,4.31,4.50,4.20,1.84,1.87,1.80,3.59,3.78,3.45,D,2.0,2.0
1521,2019-08-10,2019,Angers,Bordeaux,3,1,H,3,1,H,14,8,4,3,11,15,4,3,2,1,0,0,4,2.36,2.40,2.34,3.24,3.35,3.10,3.14,3.29,3.10,A,1.0,2.0
1146,2018-08-12,2018,Bordeaux,Strasbourg,0,2,A,0,0,D,5,17,2,6,6,12,3,10,1,0,1,0,2,1.75,1.80,1.70,4.89,5.11,4.60,3.59,3.80,3.40,A,0.0,3.0
2388,2022-01-22,2021,Lens,Marseille,0,2,A,0,1,A,9,10,3,7,7,6,8,5,3,2,0,0,2,2.55,2.60,2.50,2.71,2.75,2.65,3.46,3.63,3.30,D,2.0,2.0
600,2017-02-04,2016,Dijon,Paris SG,1,3,A,1,1,D,7,21,1,8,12,5,2,9,1,2,0,0,4,9.53,11.00,7.50,1.34,1.40,1.30,5.14,5.50,4.30,NaN,0.0,0.0
444,2016-09-24,2016,Lorient,Lyon,1,0,H,0,0,D,11,18,2,8,13,8,3,8,2,1,0,0,1,4.15,4.33,4.00,1.89,1.92,1.85,3.56,3.74,3.30,A,1.0,3.0
1579,2019-09-24,2019,Dijon,Marseille,0,0,D,0,0,D,18,9,6,2,9,14,8,3,3,3,0,0,0,3.63,3.75,3.50,2.07,2.15,2.00,3.40,3.50,3.30,A,1.0,2.0


Nombre de points moyens par match depuis le début de l'année (correspond à un classement des équipes sur la saison)

In [ ]:
df_by_team_saison = df_by_team.copy()

# Tri par Saison, Equipe et date pour calculer correctement les ordres de matchs et le cumul des points
df_by_team_saison.sort_values(by=['Saison','Equipe','Date'],inplace=True)

# Ordre du match pour chaque équipe (+1 car la numérotation commence à 0)
df_by_team_saison['Num_match'] = df_by_team_saison.groupby(['Saison','Equipe'])['Nb_points'].cumcount()+1

# Calcul du nombre de points moyens depuis le début de la saison (en prenant en compte le match de la ligne sur laquelle on se trouve)
df_by_team_saison['Nb_points_moy_par_match_saison'] = df_by_team_saison.groupby(['Saison','Equipe'])['Nb_points'].cumsum()/(df_by_team_saison.groupby(['Saison','Equipe'])['Nb_points'].cumcount()+1)

# Arrondi du nombre de points moyens à la 3eme décimale
df_by_team_saison['Nb_points_moy_par_match_saison'] = np.round(df_by_team_saison['Nb_points_moy_par_match_saison'], decimals = 3) 

Moyenne des variables sur les 5 derniers matchs (état de forme du moment)

In [ ]:
# Création d'un dictionnaire vide qui va contenir les données par équipe
dict_eq = {}

# Recherche de l'ensemble des équipes ayant été en Ligue 1 depuis 2015
equipes = pd.unique(df_by_team_saison['Equipe'])
df_by_team_saison = df_by_team_saison.sort_values(by=['Equipe','Saison','Date'])

# Pour vérification avant/après
print("Avant calcul, le dataframe df_by_team_saison contient",df_by_team_saison.shape[0],"lignes")

# Pour chaque équipe
for equipe in equipes:

  # Sélection des données d'une équipe seule
  dict_eq['df_'+equipe] = df_by_team_saison[df_by_team_saison['Equipe']==equipe]
  
  # Calcul des moyennes de chaque variable sur les 5 matchs précédents (avec un minimum de 1 match si pas 5 match disponibles)
  dict_eq['df_'+equipe]['Moy_points_5_matchs'] = dict_eq['df_'+equipe]['Nb_points'].rolling(5,closed='left',min_periods=1).mean()
  dict_eq['df_'+equipe]['Moy_buts_5_matchs'] = dict_eq['df_'+equipe]['Buts'].rolling(5,closed='left',min_periods=1).mean()
  dict_eq['df_'+equipe]['Moy_tirs_cadres_5_matchs'] = dict_eq['df_'+equipe]['Nb_tirs_cadres'].rolling(5,closed='left',min_periods=1).mean()
  dict_eq['df_'+equipe]['Moy_tirs_5_matchs'] = dict_eq['df_'+equipe]['Nb_tirs'].rolling(5,closed='left',min_periods=1).mean()
  dict_eq['df_'+equipe]['Moy_nb_fautes_5_matchs'] = dict_eq['df_'+equipe]['Nb_fautes'].rolling(5,closed='left',min_periods=1).mean()
  dict_eq['df_'+equipe]['Moy_corners_5_matchs'] = dict_eq['df_'+equipe]['Corners'].rolling(5,closed='left',min_periods=1).mean()
  dict_eq['df_'+equipe]['Moy_carton_J_5_matchs'] = dict_eq['df_'+equipe]['Carton_jaune'].rolling(5,closed='left',min_periods=1).mean()
  dict_eq['df_'+equipe]['Moy_carton_R_5_matchs'] = dict_eq['df_'+equipe]['Carton_rouge'].rolling(5,closed='left',min_periods=1).mean()

# Fusion des données de toutes les équipes
df_by_team_saison = pd.concat([dict_eq["df_Ajaccio GFCO"],dict_eq['df_Amiens'],dict_eq['df_Angers'],dict_eq['df_Bastia'],dict_eq['df_Bordeaux'],
dict_eq['df_Brest'],dict_eq['df_Caen'],dict_eq['df_Clermont'],dict_eq['df_Dijon'],dict_eq['df_Guingamp'],dict_eq['df_Lens'],dict_eq['df_Lille'],
dict_eq['df_Lorient'],dict_eq['df_Lyon'],dict_eq['df_Marseille'],dict_eq['df_Metz'],dict_eq['df_Monaco'],dict_eq['df_Montpellier'],
dict_eq['df_Nancy'],dict_eq['df_Nantes'],dict_eq['df_Nice'],dict_eq['df_Nimes'],dict_eq['df_Paris SG'],dict_eq['df_Reims'],dict_eq['df_Rennes'],
dict_eq['df_St Etienne'],dict_eq['df_Strasbourg'],dict_eq['df_Toulouse'],dict_eq['df_Troyes']])

# Vérification de la taille du dataframe 
print("Après calcul, le dataframe df_by_team_saison contient",df_by_team_saison.shape[0],"lignes")

Avant calcul, le dataframe df_by_team_saison contient 4796 lignes


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Après calcul, le dataframe df_by_team_saison contient 4796 lignes


Nombre de jours entre 2 matchs : pour le premier match de la saison, on prend le nombre de jours moyens sur tous les autres matchs de toutes les saisons sauf celle de test

In [ ]:
df_by_team_saison['Nb_jours_dernier_match']=(df_by_team_saison['Date']-np.roll(df_by_team_saison['Date'], 1)).dt.days
df_by_team_saison.loc[df_by_team_saison.Num_match==1,'Nb_jours_dernier_match']=round(df_by_team_saison.loc[(df_by_team_saison['Saison'] != '2021') & (df_by_team_saison.Num_match!=1),'Nb_jours_dernier_match'].mean())
df_by_team_saison.head()

,Equipe,Adversaire,Buts,Buts_mi_temps,Nb_tirs,Nb_tirs_cadres,Nb_fautes,Corners,Carton_jaune,Carton_rouge,Date,Saison,Resultat_match,D_E,Nb_points,Num_match,Nb_points_moy_par_match_saison,Moy_points_5_matchs,Moy_buts_5_matchs,Moy_tirs_cadres_5_matchs,Moy_tirs_5_matchs,Moy_nb_fautes_5_matchs,Moy_corners_5_matchs,Moy_carton_J_5_matchs,Moy_carton_R_5_matchs,Nb_jours_dernier_match
6,Ajaccio GFCO,Troyes,0,0,6,1,8,3,4,0,2015-08-08,2015,Nul,E,1,1,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
18,Ajaccio GFCO,Paris SG,0,0,8,2,14,1,3,0,2015-08-16,2015,Défaite,E,0,2,0.500,1.000000,0.0,1.0,6.000000,8.000000,3.000000,4.0,0.0,8
21,Ajaccio GFCO,Angers,0,0,8,3,12,9,2,0,2015-08-22,2015,Défaite,D,0,3,0.333,0.500000,0.0,1.5,7.000000,11.000000,2.000000,3.5,0.0,6
33,Ajaccio GFCO,Lille,0,0,12,6,17,4,3,0,2015-08-29,2015,Défaite,E,0,4,0.250,0.333333,0.0,2.0,7.333333,11.333333,4.333333,3.0,0.0,7
47,Ajaccio GFCO,Monaco,0,0,24,4,16,14,3,0,2015-09-13,2015,Défaite,D,0,5,0.200,0.250000,0.0,3.0,8.500000,12.750000,4.250000,3.0,0.0,15


Fusion avec le dataframe par match

In [ ]:
# Fusion du dataframe avec 1 ligne par match avec le dataframe avec 1 ligne par équipe pour récupérer le numéro du match de l'équipe à domicile
# ainsi que les données sur les 5 matchs précédents pour l'équipe à domicile
df = df.merge(right = df_by_team_saison[['Equipe','Date','Saison','Num_match','Moy_points_5_matchs','Moy_buts_5_matchs','Moy_tirs_cadres_5_matchs','Moy_tirs_5_matchs','Nb_jours_dernier_match',
'Moy_nb_fautes_5_matchs','Moy_corners_5_matchs','Moy_carton_J_5_matchs','Moy_carton_R_5_matchs']],right_on = ['Equipe','Date','Saison'],left_on = ['Equipe_domicile','Date','Saison'],how='left')
df.rename(columns={'Num_match': 'Num_match_dom',
                   'Moy_points_5_matchs':'Moy_points_5_match_dom',
                   'Moy_buts_5_matchs':'Moy_buts_5_matchs_dom',
                   'Moy_tirs_cadres_5_matchs':'Moy_tirs_cadres_5_matchs_dom',
                   'Moy_tirs_5_matchs':'Moy_tirs_5_matchs_dom',
                   'Moy_nb_fautes_5_matchs':'Moy_nb_fautes_5_matchs_dom',
                   'Moy_corners_5_matchs':'Moy_corners_5_matchs_dom',
                   'Moy_carton_J_5_matchs':'Moy_carton_J_5_matchs_dom',
                   'Moy_carton_R_5_matchs':'Moy_carton_R_5_matchs_dom',
                   'Nb_jours_dernier_match':'Nb_jours_dernier_match_dom'},inplace=True)

# Fusion du dataframe avec 1 ligne par match avec le dataframe avec 1 ligne par équipe pour récupérer le numéro du match de l'équipe à l'extérieur (peut être différent de celui à domicile (décalage de match par exemple))
# ainsi que les données sur les 5 matchs précédents pour l'équipe à l'extérieur
df = df.merge(right = df_by_team_saison[['Equipe','Date','Saison','Num_match','Moy_points_5_matchs','Moy_buts_5_matchs','Moy_tirs_cadres_5_matchs','Moy_tirs_5_matchs','Nb_jours_dernier_match',
'Moy_nb_fautes_5_matchs','Moy_corners_5_matchs','Moy_carton_J_5_matchs','Moy_carton_R_5_matchs']],right_on = ['Equipe','Date','Saison'],left_on = ['Equipe_exterieur','Date','Saison'],how='left')
df.rename(columns={'Num_match': 'Num_match_ext',
                   'Moy_points_5_matchs':'Moy_points_5_match_ext',
                   'Moy_buts_5_matchs':'Moy_buts_5_matchs_ext',
                   'Moy_tirs_cadres_5_matchs':'Moy_tirs_cadres_5_matchs_ext',
                   'Moy_tirs_5_matchs':'Moy_tirs_5_matchs_ext',
                   'Moy_nb_fautes_5_matchs':'Moy_nb_fautes_5_matchs_ext',
                   'Moy_corners_5_matchs':'Moy_corners_5_matchs_ext',
                   'Moy_carton_J_5_matchs':'Moy_carton_J_5_matchs_ext',
                   'Moy_carton_R_5_matchs':'Moy_carton_R_5_matchs_ext',
                   'Nb_jours_dernier_match':'Nb_jours_dernier_match_ext'},inplace=True)

# Suppression des variables ajoutées lors des fusions
df.drop(['Equipe_y','Equipe_x'],axis=1,inplace=True)

# Dans la dataframe df_model, nous voulons le nombre de points par macth depuis le début de la saison vu à la fin du match précédent 
# (par exemple pour le 4ème match, prendre les données à la fin du 5ème match)
# Nous ajoutons donc 1 au numéro de match dans le dataframe par équipe pour pouvoir faire la fusion correctement
df_by_team_saison['Num_match']= df_by_team_saison['Num_match']+1

# Fusion des deux dataframes par Equipe/Saison/Numéro de match recalculé pour l'équipe domicile puis extérieur
df = df.merge(right = df_by_team_saison[['Equipe','Saison','Num_match','Nb_points_moy_par_match_saison']],
left_on = ['Equipe_domicile','Saison','Num_match_dom'], right_on=['Equipe','Saison','Num_match'], how = 'left')

df.rename(columns={'Nb_points_moy_par_match_saison': 'Points_moy_match_saison_dom'},inplace=True)

# Suppression des variables ajoutées lors des fusions
df.drop(['Equipe','Num_match'],axis=1,inplace=True)

# Extérieur
df = df.merge(right = df_by_team_saison[['Equipe','Saison','Num_match','Nb_points_moy_par_match_saison']],
left_on = ['Equipe_exterieur','Saison','Num_match_ext'], right_on=['Equipe','Saison','Num_match'], how = 'left')

df.rename(columns={'Nb_points_moy_par_match_saison': 'Points_moy_match_saison_ext'},inplace=True)

# Suppression des variables ajoutées lors des fusions
df.drop(['Equipe','Num_match'],axis=1,inplace=True)



Remplacement des données manquantes

In [ ]:
# Remplacement des données manquantes de la première journée : on met des valeurs par défaut ou les moyennes hors saison 2021/2022 
# qui sera l'échantillon de test
df['Points_moy_match_saison_dom']=df['Points_moy_match_saison_dom'].fillna(1)
df['Points_moy_match_saison_ext']=df['Points_moy_match_saison_ext'].fillna(1)
df['Moy_points_5_match_dom']=df['Moy_points_5_match_dom'].fillna(1)
df['Moy_buts_5_matchs_dom']=df['Moy_buts_5_matchs_dom'].fillna(df.loc[df['Saison'] != '2021', 'Moy_buts_5_matchs_dom'].mean())
df['Moy_tirs_cadres_5_matchs_dom']=df['Moy_tirs_cadres_5_matchs_dom'].fillna(df.loc[df['Saison'] != '2021', 'Moy_tirs_cadres_5_matchs_dom'].mean())
df['Moy_tirs_5_matchs_dom']=df['Moy_tirs_5_matchs_dom'].fillna(df.loc[df['Saison'] != '2021', 'Moy_tirs_5_matchs_dom'].mean())
df['Moy_nb_fautes_5_matchs_dom']=df['Moy_nb_fautes_5_matchs_dom'].fillna(df.loc[df['Saison'] != '2021', 'Moy_nb_fautes_5_matchs_dom'].mean())
df['Moy_corners_5_matchs_dom']=df['Moy_corners_5_matchs_dom'].fillna(df.loc[df['Saison'] != '2021', 'Moy_corners_5_matchs_dom'].mean())
df['Moy_carton_J_5_matchs_dom']=df['Moy_carton_J_5_matchs_dom'].fillna(0)
df['Moy_carton_R_5_matchs_dom']=df['Moy_carton_R_5_matchs_dom'].fillna(0)
df['Moy_points_5_match_ext']=df['Moy_points_5_match_ext'].fillna(1)
df['Moy_buts_5_matchs_ext']=df['Moy_buts_5_matchs_ext'].fillna(df.loc[df['Saison'] != '2021', 'Moy_buts_5_matchs_ext'].mean())
df['Moy_tirs_cadres_5_matchs_ext']=df['Moy_tirs_cadres_5_matchs_ext'].fillna(df.loc[df['Saison'] != '2021', 'Moy_tirs_cadres_5_matchs_ext'].mean())
df['Moy_tirs_5_matchs_ext']=df['Moy_tirs_5_matchs_ext'].fillna(df.loc[df['Saison'] != '2021', 'Moy_tirs_5_matchs_ext'].mean())
df['Moy_nb_fautes_5_matchs_ext']=df['Moy_nb_fautes_5_matchs_ext'].fillna(df.loc[df['Saison'] != '2021', 'Moy_nb_fautes_5_matchs_ext'].mean())
df['Moy_corners_5_matchs_ext']=df['Moy_corners_5_matchs_ext'].fillna(df.loc[df['Saison'] != '2021', 'Moy_corners_5_matchs_ext'].mean())
df['Moy_carton_J_5_matchs_ext']=df['Moy_carton_J_5_matchs_ext'].fillna(0)
df['Moy_carton_R_5_matchs_ext']=df['Moy_carton_R_5_matchs_ext'].fillna(0)

df.head(5)

,Date,Saison,Equipe_domicile,Equipe_exterieur,Nb_buts_equipe_dom,Nb_buts_equipe_ext,Resultat_match,Nb_buts_mi_temps_equipe_dom,Nb_buts_mi_temps_equipe_ext,Resultat_mi_temps,Nb_tirs_equipe_dom,Nb_tirs_equipe_ext,Nb_tirs_cadres_equipe_dom,Nb_tirs_cadres_equipe_ext,Nb_fautes_equipe_dom,Nb_fautes_equipe_ext,Corners_equipe_dom,Corners_equipe_ext,Nb_cartons_J_equipe_dom,Nb_cartons_J_equipe_ext,Nb_cartons_R_equipe_dom,Nb_cartons_R_equipe_ext,Nb_total_but_match,Moy_cote_domicile,Max_cote_domicile,Min_cote_domicile,Moy_cote_exterieur,Max_cote_exterieur,Min_cote_exterieur,Moy_cote_nul,Max_cote_nul,Min_cote_nul,Resultat_match_an_dernier,Nb_buts_equipe_dom_an_dernier,Nb_buts_equipe_ext_an_dernier,Num_match_dom,Moy_points_5_match_dom,Moy_buts_5_matchs_dom,Moy_tirs_cadres_5_matchs_dom,Moy_tirs_5_matchs_dom,Nb_jours_dernier_match_dom,Moy_nb_fautes_5_matchs_dom,Moy_corners_5_matchs_dom,Moy_carton_J_5_matchs_dom,Moy_carton_R_5_matchs_dom,Num_match_ext,Moy_points_5_match_ext,Moy_buts_5_matchs_ext,Moy_tirs_cadres_5_matchs_ext,Moy_tirs_5_matchs_ext,Nb_jours_dernier_match_ext,Moy_nb_fautes_5_matchs_ext,Moy_corners_5_matchs_ext,Moy_carton_J_5_matchs_ext,Moy_carton_R_5_matchs_ext,Points_moy_match_saison_dom,Points_moy_match_saison_ext
0,2015-08-07,2015,Lille,Paris SG,0,1,A,0,0,D,12,7,2,2,20,17,3,2,3,2,0,1,1,6.22,7.00,4.65,1.65,1.75,1.57,3.63,3.80,3.40,NaN,0.0,0.0,1,1.0,1.29075,4.122589,11.9191,8,13.019501,4.765234,0.0,0.0,1,1.0,1.322451,4.190211,12.128774,8,12.97336,4.873537,0.0,0.0,1.0,1.0
1,2015-08-08,2015,Bastia,Rennes,2,1,H,0,1,A,11,6,3,3,15,19,6,2,0,1,1,0,3,2.56,2.71,2.30,2.97,3.10,2.80,3.01,3.10,2.88,NaN,0.0,0.0,1,1.0,1.29075,4.122589,11.9191,8,13.019501,4.765234,0.0,0.0,1,1.0,1.322451,4.190211,12.128774,8,12.97336,4.873537,0.0,0.0,1.0,1.0
2,2015-08-08,2015,Marseille,Caen,0,1,A,0,1,A,17,8,4,4,10,10,14,2,1,0,0,0,1,1.55,1.57,1.50,6.05,6.59,5.50,4.08,4.40,3.70,NaN,0.0,0.0,1,1.0,1.29075,4.122589,11.9191,8,13.019501,4.765234,0.0,0.0,1,1.0,1.322451,4.190211,12.128774,8,12.97336,4.873537,0.0,0.0,1.0,1.0
3,2015-08-08,2015,Montpellier,Angers,0,2,A,0,1,A,11,16,4,6,12,15,5,9,3,2,0,0,2,1.78,1.83,1.75,5.09,5.53,4.35,3.33,3.50,3.10,NaN,0.0,0.0,1,1.0,1.29075,4.122589,11.9191,8,13.019501,4.765234,0.0,0.0,1,1.0,1.322451,4.190211,12.128774,8,12.97336,4.873537,0.0,0.0,1.0,1.0
4,2015-08-08,2015,Nantes,Guingamp,1,0,H,0,0,D,9,10,4,4,13,16,6,2,2,2,0,0,1,2.17,2.25,2.10,3.68,4.20,3.40,3.07,3.15,2.90,NaN,0.0,0.0,1,1.0,1.29075,4.122589,11.9191,8,13.019501,4.765234,0.0,0.0,1,1.0,1.322451,4.190211,12.128774,8,12.97336,4.873537,0.0,0.0,1.0,1.0


Ajout du poucentage de buts / tirs cadrés sur 5 matchs

In [ ]:
df['Pct_buts_par_tirs_cadres_5m_dom']=df['Moy_buts_5_matchs_dom']/df['Moy_tirs_cadres_5_matchs_dom']
df['Pct_buts_par_tirs_cadres_5m_ext']=df['Moy_buts_5_matchs_ext']/df['Moy_tirs_cadres_5_matchs_ext']

Fusion avec le dataframe précédent pour mettre les notes des équipes à domicile et à l'extérieur

In [ ]:
# Domicile
df = df.merge(right = df_note[['Equipe','Saison',
                               'note_att','note_mil',
                               'note_def', 'note_gen']],left_on = ['Equipe_domicile','Saison'],right_on = ['Equipe','Saison'],how='left')

df.rename(columns={'note_att':'note_att_dom',
                   'note_mil':'note_mil_dom',
                   'note_def':'note_def_dom',
                   'note_gen':'note_gen_dom',},inplace=True)

# Extérieur
df = df.merge(right = df_note[['Equipe','Saison',
                               'note_att','note_mil',
                               'note_def', 'note_gen']],left_on = ['Equipe_exterieur','Saison'],right_on = ['Equipe','Saison'],how='left')

df.rename(columns={'note_att':'note_att_ext',
                   'note_mil':'note_mil_ext',
                   'note_def':'note_def_ext',
                   'note_gen':'note_gen_ext',},inplace=True)

Mise en forme du résultat de l'an dernier au format numérique (dummies)

In [ ]:
df = pd.get_dummies(df, columns=["Resultat_match_an_dernier"], prefix = ["Res_an_dernier"])

Création d'un index et du daraframe final

In [ ]:
# Création d'un index unique
df.index = df.Saison + ' - ' + df.Equipe_domicile + ' - ' + df.Equipe_exterieur

# Suppression des variables qui ont servi à la création des variables connues avant le match
df_model= df.drop(['Date','Equipe_domicile','Equipe_exterieur','Nb_buts_equipe_dom','Nb_buts_equipe_ext','Nb_buts_mi_temps_equipe_dom',
                   'Nb_buts_mi_temps_equipe_ext','Resultat_mi_temps', 'Nb_tirs_equipe_dom','Nb_tirs_equipe_ext','Nb_tirs_cadres_equipe_dom',
                   'Nb_tirs_cadres_equipe_ext','Nb_fautes_equipe_dom','Nb_fautes_equipe_ext','Corners_equipe_dom','Corners_equipe_ext',
                   'Nb_cartons_J_equipe_dom','Nb_cartons_J_equipe_ext','Nb_cartons_R_equipe_dom','Nb_cartons_R_equipe_ext','Nb_total_but_match',
                   'Equipe_y','Equipe_x'],axis=1)


In [ ]:
# Export du csv df_by_team dans google colab pour réutilisation dans le feature engineering
df_model.to_csv('/content/df_model.csv')